<a href="https://colab.research.google.com/github/FabioNicotra/FabiosPortfolio/blob/main/01_GeneticAlgorithm/GeneticAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A silly genetic algorithm
This project was assigned during the course *Data Structures in Python* held by Politecnico di Torino in cooperation with Reply IT.

In computer science and operations research, a genetic algorithm is a metaheuristic inspired by the process of natural selection. They are part of a larger families of algorithms known as evolutionary algorithms. Genetic algorithms rely on the existence of a candidates population that evolves in time, exploiting operators such as mutation, crossover and selection, in order to generate high-quality solutions to optimization and search problems.

Genetic algorithms *optimize*, i.e., they select the best solution according to given criteria from a set of available alternatives. They efficiently search the solution space thoroughly enough to avoid picking a good answer when a better one is available. They do not perform exhaustive searches of the space, since they don't try every possible solution. They continuously grade solutions and then use them to make choices going forward.

Genetic algorithms use a *fitness function* to preserve/discard candidates in a population, and then build the next iteration of such a population according to context rules.

The basic process is as follows:
1. Randomly generate a population of solutions
2. Measure the fitness of each solution
3. Select the best solutions and discard the rest
4. Cross over elements in the best solutions to make new solutions
5. Mutate a small number of elements in the solutions by changing their value
6. Return to step 2 and repeat until converge or halt condition.

Halting conditions could either be having found a *good enough* solution or having exhausted all our resources, such as number of iterations or reaching a time deadline.


## The smartest dogs in the world (sort of...)
We strive to algorithmically generate a breed of smarter dogs, i.e., generate a population of dogs with an average smart factor higher than a given threshold, starting from a population way duller than our target.

Let's say for the sake of simplicity that our population has a 50/50 male/female ratio. At each iteration crossover happens, generating new offspring. Most likely the new offspring present the same characteristics of their parents, so we apply mutation to a very limited number of them. Before moving on, we resize our population in order to keep only the smartest individuals. We make sure to keep the same initial gender ratio and the same number of initial individuals. The whole process then becomes a big repeating loop, until convergence or stop.

### Parametrizing our experiments
Here follows a set of constants that we can use to determine how our algorithm will evolve. You can play with these number a see different outcomes for your runs.

In [8]:
TARGET = 100 # Target "smartness" for our population
POP_SIZE = 40 # The size of the population at the start of each iteration (keep this even, if you change it)
INIT_MIN_SMART = 1 # The minimum smartness of a dog at ITER 0
INIT_MAX_SMART = 10 # The maximum smartness of a dog at ITER 0
INIT_MODE_SMART = 3 # The most comon smartness value of a dog at ITER 0
MUTATE_ODDS = 0.01 # Probability of mutation for a new puppy
MUTATE_MIN_FACTOR = 0.4 # Lower bound for the the product factor we multuply smartness for, if mutation happens
MUTATE_MAX_FACTOR = 1.3 # Upper bound for the the product factor we multuply smartness for, if mutation happens
LITTER_SIZE = 5 # Puppies born from each pair of adult dogs
MAX_ITER = 1000 # Max number of iterations

### Generate the initial population
We want to generate an initial population of a fixed size with a 50/50 gender split ratio, of dogs with a given initial smartness factor according to parameters. The `random.triangular()` function generates random values in a range with a skew toward a wanted mode. In this first implementation, no information about the gender is assigned because of the 50/50 split ratio assumption.

In [9]:
import random

def populate():
    pop = [random.triangular(INIT_MIN_SMART, INIT_MAX_SMART, INIT_MODE_SMART) for _ in range(POP_SIZE)]
    return pop

### Measure the fitness of the population
The goodness of our population is evaluated as the average of the smartness of the individuals with respect to our target value. This function in used as an halting criteria for the main loop.
Functions from the `statistics` module are used for this purpose.

In [10]:
import statistics

def fitness(pop):
    avg = statistics.mean(pop)
    std = statistics.stdev(pop)
    return avg, std

### Breed the next generation of dogs
We have to generate the new offspring. As a rule, we assume that given a pair of parent dogs, a new puppy smartness will lie in that same min-max range, unless mutation occours.
In order to replicate what would happen in nature, we should not assume that smarter dogs pair with smarter dogs. We use `random.shuffle()` on our collection of dogs before pairing them. Each pair breed as many puppies as the litter size parameter indicates.

In [11]:
def breed(pop):
    random.shuffle(pop)
    puppies = []
    for i in range(0,len(pop),2):
        parents_avg = 0.5*(pop[i]+pop[i+1])
        puppies += [random.triangular(pop[i], pop[i+1], parents_avg) for _ in range(LITTER_SIZE)]
    return puppies

### Select the best candidates
At each iteration, we have to resize our current population down to its initial size, preserving only the smartest dogs available for the next iteration. ***We have to make sure to keep also the initial gender split ratio. ***

In [12]:
def select(pop,puppies):
    families = pop + puppies
    families.sort(reverse=True)
    newGen = families[0:POP_SIZE]
    return newGen

### Mutate the set of offspring
A small percentage of the puppies should undergo mutation. This could either lead to a lower or a higher smartness value, since their base value is directly multiplied with a random value in the closed range `[MUTATE_MIN_FACTOR, MUTATE_MAX_FACTOR]`.

In [13]:
from math import floor

def mutate(puppies):
    size = len(puppies)
    mutants = floor(size*MUTATE_ODDS)
    if mutants == 0:
        mutants = random.randint(0,1)
    random.shuffle(puppies)
    for i in range(mutants):
        puppies[i] = puppies[i]*random.uniform(MUTATE_MIN_FACTOR,MUTATE_MAX_FACTOR)

### Put it all together
The `main()` function manages the other functions and the primary evolutionary loop. It is also responsible for printing all the relevant information and outputs of our experiment.

In [14]:
def main():
    dogs = populate()
    avg, std = fitness(dogs)
    n_iter = 0
    while avg < TARGET and n_iter < MAX_ITER:
        if n_iter > 0:
            litter = breed(dogs)
            mutate(litter)
            dogs = select(dogs, litter)
        avg, std = fitness(dogs)
        print(f"Iter {n_iter}, avg = {avg:.2f}, std = {std:.2f}, min = {min(dogs):.2f}, max = {max(dogs):.2f}")
        n_iter += 1

if __name__ == '__main__':
    main()

Iter 0, avg = 4.42, std = 1.76, min = 1.51, max = 8.52
Iter 1, avg = 6.41, std = 0.98, min = 5.02, max = 8.52
Iter 2, avg = 7.51, std = 0.48, min = 6.95, max = 8.70
Iter 3, avg = 8.02, std = 0.19, min = 7.77, max = 8.70
Iter 4, avg = 8.23, std = 0.14, min = 8.07, max = 8.70
Iter 5, avg = 8.39, std = 0.10, min = 8.27, max = 8.70
Iter 6, avg = 8.48, std = 0.05, min = 8.43, max = 8.70
Iter 7, avg = 8.54, std = 0.05, min = 8.50, max = 8.70
Iter 8, avg = 8.59, std = 0.03, min = 8.55, max = 8.70
Iter 9, avg = 8.62, std = 0.02, min = 8.60, max = 8.70
Iter 10, avg = 8.64, std = 0.02, min = 8.62, max = 8.70
Iter 11, avg = 8.65, std = 0.01, min = 8.64, max = 8.70
Iter 12, avg = 8.73, std = 0.41, min = 8.66, max = 11.24
Iter 13, avg = 8.96, std = 0.72, min = 8.67, max = 11.24
Iter 14, avg = 9.68, std = 0.66, min = 8.69, max = 11.24
Iter 15, avg = 10.48, std = 0.41, min = 9.99, max = 12.33
Iter 16, avg = 10.77, std = 0.36, min = 10.53, max = 12.33
Iter 17, avg = 11.15, std = 0.37, min = 10.74, max

## Object-oriented revision
In this section, I present a refined implementation of the previous problem using an object-oriented paradigm. By leveraging the power of object-oriented programming, this revised code offers enhanced modularity, reusability, and maintainability. The solution is designed around intuitive classes and methods, encapsulating the functionality and data structures necessary to tackle the problem efficiently.